In [1]:
import src.data_prep
import librosa
import librosa.display
import soundfile as sf
import IPython.display as ipd

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

%matplotlib inline


In [2]:
from keras import backend as K
from keras.models import Model
from keras.layers import (Input, Lambda)
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint 

In [3]:
import random
import _pickle as pickle
from sklearn.model_selection import train_test_split


# Organizing paths to audio and text files

In [3]:
audio_paths, texts = src.data_prep.get_audio_and_text_data('/Users/joshbernd/Desktop/gal_notes/Capstone/Speech_Detection/data/LibriSpeech_train/train-clean-100/')        


text file:  0
text file:  50
text file:  100
text file:  150
text file:  200
text file:  250
text file:  300
text file:  350
text file:  400
text file:  450
text file:  500
text file:  550


Here is how the data looks:

In [4]:
audio_paths[111]#, texts[111]

'/Users/joshbernd/Desktop/gal_notes/Capstone/Speech_Detection/data/LibriSpeech_train/train-clean-100/8580/287364/8580-287364-0003.flac'

## Let's store our phonemes and text for safe keeping

with open("data/ipa.txt", "w") as f:
    for t in texts:
        f.write(str(t) +"\n")

with open("data/audio_path.txt", "w") as f:
    for a in audio_paths:
        f.write(str(a) + "\n")

In [36]:
## reading in saved texts
texts = []
with open("data/ipa.txt", "r") as f:
  for line in f:
    texts.append(line.strip())


In [4]:
audio_list = []
with open('data/audio_path.txt', 'r') as f:
    for line in f:
        audio_list.append(line.strip())

In [5]:
# find all elements in text list:
unique = []
for t in texts:
    for char in t:
        if char not in unique:
            unique.append(char)
            
unique

['h',
 'æ',
 'd',
 ' ',
 'l',
 'e',
 'ɪ',
 'ˌ',
 'b',
 'i',
 'ˈ',
 'f',
 'ɔ',
 'r',
 'ə',
 'p',
 'ɛ',
 'v',
 't',
 'n',
 'z',
 'a',
 'ʊ',
 'k',
 's',
 'j',
 'm',
 'o',
 'ð',
 'u',
 'w',
 'ɑ',
 'ŋ',
 'g',
 'θ',
 'ʧ',
 'c',
 '*',
 'ʃ',
 'ʤ',
 'ʒ',
 'y',
 "'",
 'x',
 'q']

In [10]:
texts[0], audio_paths[0]

('hæd leɪd ˌbiˈfɔr hər ə pɛr əv ɔlˈtərnətɪvz naʊ əv kɔrs jʊr kəmˈplitli jʊr oʊn ˈmɪstrəs ənd ər ɛz fri ɛz ðə bərd ɔn ðə baʊ aɪ doʊnt min ju wər nɑt soʊ ˌbiˈfɔr bət jʊr æt ˈprɛzənt ɔn ə ˈdɪfərənt ˈfʊtɪŋ',
 '/Users/joshbernd/Desktop/gal_notes/Capstone/Speech_Detection/data/LibriSpeech_train/train-clean-100/1069/133709/1069-133709-0000.flac')

## Trying to get the rnn to work

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [72]:
model = keras.Sequential()
model.add(layers.Embedding(input_dim=161, output_dim=47))

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(layers.GRU(47, return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(layers.SimpleRNN(20))

model.add(layers.Dense(10))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 47)          7567      
_________________________________________________________________
gru_2 (GRU)                  (None, None, 47)          13536     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 20)                1360      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
Total params: 22,673
Trainable params: 22,673
Non-trainable params: 0
_________________________________________________________________


In [15]:
from keras.models import Model
from keras.layers import Input, GRU, Activation

from keras import backend as K
from keras.models import Model
from keras.layers import (Input, Lambda)
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint   

In [113]:
def simple_rnn_model(input_dim, output_dim=47):
    """ Build a recurrent network for speech 
    """
    # Main acoustic input
    input_data = Input(name='the_input', shape=(None, input_dim))
    print(input_data.shape)
    # Add recurrent layer
    simp_rnn = GRU(output_dim, return_sequences=True, 
                 implementation=2, name='rnn')(input_data)
    # Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(simp_rnn)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: x
    print(model.summary())
    return model

In [114]:
model_0 = simple_rnn_model(input_dim=161)

(None, None, 161)
Model: "functional_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 161)]       0         
_________________________________________________________________
rnn (GRU)                    (None, None, 47)          29610     
_________________________________________________________________
softmax (Activation)         (None, None, 47)          0         
Total params: 29,610
Trainable params: 29,610
Non-trainable params: 0
_________________________________________________________________
None


In [115]:
train_model(input_to_softmax=model_0, 
            pickle_path='model_0.pickle', 
            save_model_path='model_0.h5',
           )

entered load_train_data
entered load_metadata
loaded metadata
entered fit_train
fitted metadata
entered load_validation_data
entered load_metadata
entered fit_train
validation_steps:  1
initialized ctc model
compiled model
made sure results path exists
calling model.fit_generator


TypeError: fit() got an unexpected keyword argument 'generator'

## Let's develop training pipeline and AudioGenerator Class

In [9]:
from src.data_prep import calc_feat_dim, spectrogram_from_file, text_to_int_sequence

In [108]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def add_ctc_loss(input_to_softmax):
    the_labels = Input(name='the_labels', shape=(None,), dtype='float32')
    input_lengths = Input(name='input_length', shape=(1,), dtype='int64')
    label_lengths = Input(name='label_length', shape=(1,), dtype='int64')
    output_lengths = Lambda(input_to_softmax.output_length)(input_lengths)
    # CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')(
            [input_to_softmax.output, the_labels, output_lengths, label_lengths])
    model = Model(
    inputs=[input_to_softmax.input, the_labels, input_lengths, label_lengths], 
    outputs=loss_out)
    return model




def train_model(input_to_softmax, 
                pickle_path, 
                save_model_path, 
                train_data='data/audio_path.txt',
                validation_data="data/ipa.txt", 
                minibatch_size=20,
                optimizer=SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
                epochs=1,
                verbose=1,):
    
    ## create audio_generator instance to get batches of audio
    audio_generator = AudioGenerator(minibatch_size=minibatch_size,)
    audio_generator.load_train_data()      # went here
    audio_generator.load_validation_data() # went here
    # calculate steps_per_epoch
    num_train_examples=len(audio_generator.train_audio)
    steps_per_epoch = num_train_examples//minibatch_size
    # calculate validation_steps
    num_valid_samples = len(audio_generator.valid_audio) 
    validation_steps = num_valid_samples//minibatch_size
    print('validation_steps: ', validation_steps)
    # add CTC loss to the NN specified in input_to_softmax
    model = add_ctc_loss(input_to_softmax)
    print('initialized ctc model')
    
    
    # CTC loss is implemented elsewhere, so use a dummy lambda function for the loss
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    print('compiled model')
    
    # make results/ directory, if necessary
    if not os.path.exists('results'):
        os.makedirs('results')
    print('made sure results path exists')    
    # add checkpointer
    checkpointer = ModelCheckpoint(filepath='results/'+save_model_path, 
                                   verbose=0)
    
    # train model
    print('calling model.fit_generator')
    hist = model.fit(generator=audio_generator.next_train(),
                     steps_per_epoch=steps_per_epoch,
                     epochs=epochs,
                     validation_data=audio_generator.next_valid(),
                     validation_steps=validation_steps,
                     callbacks=[checkpointer], 
                     verbose=verbose)
    print('saving model loss')
    
    # save model loss
    with open('results/'+pickle_path, 'wb') as f:
        pickle.dump(hist.history, f)


In [109]:
## make an audio generator
import warnings
warnings.filterwarnings("ignore")
                        
RNG_SEED = 123                        
class AudioGenerator():
    def __init__(self, step=10, window=20, max_freq=8000, mfcc_dim=13,
        minibatch_size=20, desc_file=None,):
        
        """
        Params:
            step (int): Step size in milliseconds between windows (for spectrogram ONLY)
            window (int): FFT window size in milliseconds (for spectrogram ONLY)
            max_freq (int): Only FFT bins corresponding to frequencies between
                [0, max_freq] are returned (for spectrogram ONLY)
            desc_file (str, optional): Path to a JSON-line file that contains
                labels and paths to the audio files. If this is None, then
                load metadata right away
        """
    
        self.feat_dim = calc_feat_dim(window, max_freq)
        self.feats_mean = np.zeros((self.feat_dim))
        self.feats_std = np.ones((self.feat_dim,))
        self.step = step
        self.window = window
        self.max_freq = max_freq
        self.rng = random.Random(RNG_SEED)
        self.cur_train_index = 0
        self.cur_valid_index = 0
        self.cur_test_index = 0
        self.minibatch_size = minibatch_size
    
    def get_batch(self, partition):
        print('entered get_batch')
        if partition == 'train':
            audio_paths = self.train_audio
            cur_index = self.cur_train_index
            texts = self.train_texts
        elif partition == 'valid':
            audio_paths = self.valid_audio
            cur_index = self.cur_valid_index
            print('cur_index: ', cur_index)
            texts = self.valid_texts
#         elif partition == 'test':
#             audio_paths = self.test_audio_paths
#             cur_index = self.test_valid_index
#             texts = self.test_texts
            
        features = [self.normalize(self.featurize(a)) for a in audio_paths[cur_index:cur_index+self.minibatch_size]]

#         calculate necessary sizes
        max_length = max([features[i].shape[0] for i in range(0, self.minibatch_size)])
        max_string_length = max([len(texts[cur_index+i]) for i in range(0, self.minibatch_size)])     
        
        
        X_data = np.zeros([self.minibatch_size, max_length, self.feat_dim])
        labels = np.ones([self.minibatch_size, max_string_length]) * 47
        
        
#         X_data = np.zeros([self.minibatch_size, self.feat_dim])
#         labels = np.ones([self.minibatch_size]) * 47
                
        
        input_length = np.zeros([self.minibatch_size, 1])
        label_length = np.zeros([self.minibatch_size, 1])
        
        for i in range(0, self.minibatch_size):
            # get each featurized/normalized spectrogram
            feat = features[i]
            input_length[i] = feat.shape[0]
            X_data[i, :feat.shape[0], :] = feat
            
            #calc labels and lable length
            label = np.array(text_to_int_sequence(texts[cur_index+i]))
            labels[i, :len(label)] = label
            label_length[i] = len(label)
        print('shape of X_data: ', X_data.shape)   
        print('shape of labels: ', labels.shape)
        #return the arrays
        outputs = {'ctc': np.zeros([self.minibatch_size])}
        inputs = {'the_input': X_data,
                 'the labels': labels, 
                 'input_length': input_length, 
                 'label_length': label_length
                 }
        return (inputs, outputs)

    
    def shuffle_data_by_partition(self, partition):
        print('entered shuffle_data_by_partition')
        """ Shuffle the training or validation data
        """
        if partition == 'train':
            self.train_audio, self.train_texts = shuffle_data(
                self.train_audio, self.train_texts)
#         elif partition == 'valid':
#             self.valid_audio_paths, self.valid_durations, self.valid_texts = shuffle_data(
#                 self.valid_audio_paths, self.valid_durations, self.valid_texts)
#         else:
#             raise Exception("Invalid partition. "
#                 "Must be train/validation")
        
  
    def fit_train(self, k_samples=100):
        print('entered fit_train')
        """ Estimate the mean and std of the features from the training set
        Params:
            k_samples (int): Use this number of samples for estimation
        """
        k_samples = min(k_samples, len(self.train_audio))
        samples = self.rng.sample(self.train_audio, k_samples)
        feats = [self.featurize(s) for s in samples]
        feats = np.vstack(feats)
        self.feats_mean = np.mean(feats, axis=0)
        self.feats_std = np.std(feats, axis=0)


    def featurize(self, audio_clip):
#         print('entered featurize')
        """ For a given audio clip, calculate the corresponding feature
        Params:
            audio_clip (str): Path to the audio clip
        """
        
        return spectrogram_from_file(audio_clip, 
                                     step=self.step, 
                                     window=self.window,
                                     max_freq=self.max_freq)


    def normalize(self, feature, eps=1e-14):
#         print('entered normalize')
        """ Center a feature using the mean and std
        Params:
            feature (numpy.ndarray): Feature to normalize
        """
        return (feature - self.feats_mean) / (self.feats_std + eps)
    
    def load_train_data(self):
        print('entered load_train_data')
        self.load_metadata('train')
        print('loaded metadata')
        self.fit_train()
        print('fitted metadata')

    def load_validation_data(self):
        print('entered load_validation_data')
        self.load_metadata('validation')
        self.fit_train()
                
                    
    def load_metadata(self, partition):
        print('entered load_metadata')
        audio_list = []
        texts = []

        with open('data/audio_path.txt', 'r') as f:
            for line in f:
                audio_list.append(line.strip())
        with open("data/ipa.txt", "r") as f:
          for line in f:
            texts.append(line.strip())
            
        ## testing on 100 samples to make sure it works
        audio_list = audio_list[:100]
        texts = texts[:100]

        X_train, X_val, y_train, y_val = train_test_split(audio_list, 
                                                          texts, 
                                                          test_size=0.33, 
                                                          random_state=42)        
        if partition == 'train':
            self.train_audio = X_train
            self.train_texts = y_train
        elif partition == 'validation':
            self.valid_audio = X_val
            self.valid_texts = y_val
        
        
    def next_train(self):
        '''
        get next batch of training data
        '''
        print('entered next_train')
        i = 0
        while True:
            print(i)
            ret = self.get_batch('train')
            self.cur_train_index += self.minibatch_size
            if self.cur_train_index >= len(self.train_texts) - self.minibatch_size:
                self.cur_train_index = 0
                self.shuffle_data_by_partition('train')
            i += 1
            yield ret
        
        
    def next_valid(self):
        '''
        get next batch of validation data
        '''
        print('entered next_valid')
        while True:
            ret = self.get_batch('valid')
            self.cur_valid_index += self.minibatch_size
            if self.cur_valid_index >= len(self.valid_texts) - self.minibatch_size:
                self.cur_valid_index = 0
                self.shuffle_data_by_partition('valid')
            yield ret
        
        
def shuffle_data(audio_paths, texts):
    print('entered shuffle_data')
    """ Shuffle the data (called after making a complete pass through 
        training or validation data during the training process)
    Params:
        audio_paths (list): Paths to audio clips
        durations (list): Durations of utterances for each audio clip
        texts (list): Sentences uttered in each audio clip
    """
    p = np.random.permutation(len(audio_paths))
    audio_paths = [audio_paths[i] for i in p] 
    texts = [texts[i] for i in p]
    return audio_paths, texts





                
                


In [103]:
audiogen = AudioGenerator()

In [104]:
np.ones([20]) * 47
label = np.array(text_to_int_sequence(texts[cur_index+i]))

NameError: name 'cur_index' is not defined

In [105]:
texts[0]

'hæd leɪd ˌbiˈfɔr hər ə pɛr əv ɔlˈtərnətɪvz naʊ əv kɔrs jʊr kəmˈplitli jʊr oʊn ˈmɪstrəs ənd ər ɛz fri ɛz ðə bərd ɔn ðə baʊ aɪ doʊnt min ju wər nɑt soʊ ˌbiˈfɔr bət jʊr æt ˈprɛzənt ɔn ə ˈdɪfərənt ˈfʊtɪŋ'

In [12]:
char_map_str = """

' 0
<SPACE> 1
'h' 2 
'æ' 3 
'd' 4 
'q' 5 
'l' 6 
'e' 7 
'ɪ' 8 
'ˌ' 9 
'b' 10 
'i' 11 
'ˈ' 12 
'f' 13 
'ɔ' 14 
'r' 15 
'ə' 16 
'p' 17 
'ɛ' 18 
'v' 19 
't' 20  
'n' 21
'z' 22
'a' 23
'ʊ' 24
'k' 25
's' 26
'j' 27
'm' 28
'o' 29 
'ð' 30 
'u' 31 
'w' 32 
'ɑ' 33 
'ŋ' 34 
'g' 35 
'θ' 36 
'ʧ' 37 
'c' 38 
'*' 39 
'ʃ' 40 
'ʤ' 41 
'ʒ' 42 
'y' 43 
"'" 44 
'x' 45 
 

"""

char_map = {}
index_map = {}
for line in char_map_str.strip().split('\n'):
    ch, index = line.split()
    char_map[ch] = int(index)
    index_map[int(index)+1] = ch
index_map[2] = ' '

In [13]:
char_map

{"'": 0,
 '<SPACE>': 1,
 "'h'": 2,
 "'æ'": 3,
 "'d'": 4,
 "'q'": 5,
 "'l'": 6,
 "'e'": 7,
 "'ɪ'": 8,
 "'ˌ'": 9,
 "'b'": 10,
 "'i'": 11,
 "'ˈ'": 12,
 "'f'": 13,
 "'ɔ'": 14,
 "'r'": 15,
 "'ə'": 16,
 "'p'": 17,
 "'ɛ'": 18,
 "'v'": 19,
 "'t'": 20,
 "'n'": 21,
 "'z'": 22,
 "'a'": 23,
 "'ʊ'": 24,
 "'k'": 25,
 "'s'": 26,
 "'j'": 27,
 "'m'": 28,
 "'o'": 29,
 "'ð'": 30,
 "'u'": 31,
 "'w'": 32,
 "'ɑ'": 33,
 "'ŋ'": 34,
 "'g'": 35,
 "'θ'": 36,
 "'ʧ'": 37,
 "'c'": 38,
 "'*'": 39,
 "'ʃ'": 40,
 "'ʤ'": 41,
 "'ʒ'": 42,
 "'y'": 43,
 '"\'"': 44,
 "'x'": 45}